In [3]:
import pandas as pd 
import numpy as np
import dask.dataframe as dd

# parallelize the work using dask
from dask.distributed import Client
client = Client()

#for not having the output truncated
n=100 #number of rows to print
pd.set_option('display.max_rows', n)

In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 3.72 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35705,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 3.72 GiB
Comm: tcp://127.0.0.1:34879,Total threads: 2
Dashboard: http://127.0.0.1:45119/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:37609,


In [10]:
#loading data 

path = "./Z0.02_a3_cM70/output_0.csv"
#"/home/albertos/Astro_data/demo_Z0.00014/output_*.csv"
df0 = dd.read_csv(path, dtype={'BEvent': float})
df0 = df0.persist()

### Selecting systems

In [6]:
#### Wolf Rayet - Black hole system
cond_1 = ((df0.PhaseBSE_0==8) | (df0.PhaseBSE_0==7)) & (df0.PhaseBSE_1==14.0) 
cond_2 = ((df0.PhaseBSE_1==8) | (df0.PhaseBSE_1==7)) & (df0.PhaseBSE_0==14.0)
df0_WRBH = df0.loc[ cond_1 | cond_2]

data_WRBH = df0_WRBH[['ID', 'Mass_0', 'Mass_1', 'Semimajor', 'Eccentricity']].drop_duplicates('ID') #selecting only 1st row of each ID
data_WRBH.compute()

id0_WRBH = data_WRBH['ID'] #taking only the IDs

2024-05-17 11:38:35,343 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:37081' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('drop-duplicates-chunk-54dca2048a0ecc500e6c95db2077a0f6-cf47a67bc1754a3f5306bcd6bb7465e3', 979), ('read-csv-7a011b273be2ec3956ae50b59d22a75b', 998), ('read-csv-7a011b273be2ec3956ae50b59d22a75b', 949), ('drop-duplicates-chunk-54dca2048a0ecc500e6c95db2077a0f6-cf47a67bc1754a3f5306bcd6bb7465e3', 982), ('drop-duplicates-chunk-54dca2048a0ecc500e6c95db2077a0f6-cf47a67bc1754a3f5306bcd6bb7465e3', 994), ('read-csv-7a011b273be2ec3956ae50b59d22a75b', 958), ('read-csv-7a011b273be2ec3956ae50b59d22a75b', 967), ('read-csv-7a011b273be2ec3956ae50b59d22a75b', 982), ('read-csv-7a011b273be2ec3956ae50b59d22a75b', 979), ('drop-duplicates-chunk-54dca2048a0ecc500e6c95db2077a0f6-cf47a67bc1754a3f5306bcd6bb7465e3', 966), ('read-csv-7a011b273be2ec3956ae50b59d22a75b', 994), ('drop-duplicates-chunk-54dca2048a0ecc500e6c95db2077a0

2024-05-17 11:38:35,810 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:37081
Traceback (most recent call last):
  File "/home/gaia/.local/lib/python3.10/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/home/gaia/.local/lib/python3.10/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/worker.py", line 2059, in gather_dep
    response = await get_data_from_worker(
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/worker.py", line 2863, in get_data_from_worker
    response = await send_recv(
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/core.py", line 1154, in 

In [7]:
#### GW BBH merge (Binary Black Holes that merge via Gravitational Waves Emission in less than 14 bilions years)
cond_1 = (df0.PhaseBSE_0 == 14.0) & (df0.PhaseBSE_1 == 14.0)
cond_2 = (df0.GWtime + df0.BWorldtime) < int(14e+03)
id0_GW = df0.loc[cond_1 & cond_2]['ID'].drop_duplicates()
id0_GW.compute()

2024-05-17 11:49:14,519 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:39981 -> tcp://127.0.0.1:44613
Traceback (most recent call last):
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/worker.py", line 1783, in get_data
    response = await comm.read(deserializers=serializers)
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/comm/tcp.py", line 236, in read
    convert_stream_closed_error(self, e)
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/comm/tcp.py", line 142, in convert_stream_closed_error
    raise CommClosedError(f"in {obj}: {exc}") from exc
distributed.comm.core.CommClosedEr

248003      10902
208213      24959
193250      28453
252672      62081
178980      79808
           ...   
3581      4104223
58877     4108713
216464    4110843
169190    4124692
69442     4130734
Name: ID, Length: 340, dtype: int64

In [8]:
id0_both = np.intersect1d(id0_WRBH, id0_GW) #ID of WR-BH systems that evolve in a BH-BH systems that merge via GW emission

2024-05-17 12:11:40,455 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:43265 -> tcp://127.0.0.1:41613
Traceback (most recent call last):
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/worker.py", line 1783, in get_data
    response = await comm.read(deserializers=serializers)
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/comm/tcp.py", line 236, in read
    convert_stream_closed_error(self, e)
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/comm/tcp.py", line 142, in convert_stream_closed_error
    raise CommClosedError(f"in {obj}: {exc}") from exc
distributed.comm.core.CommClosedEr

KeyboardInterrupt: 

Process Dask Worker process (from Nanny):
2024-05-17 12:15:44,259 - distributed.nanny - ERROR - Worker process died unexpectedly
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/process.py", line 202, in _run
    target(*args, **kwargs)
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/nanny.py", line 1019, in _run
    asyncio_run(run(), loop_factory=get_loop_factory())
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/compatibility.py", line 236, in asyncio_run
    return loop.run_until_complete(main)
  File "/usr/lib/python3.10/asyncio/base_events.py", line 636, in run_until_complete
    self.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
   

In [ ]:
print('Binary systems of a Wolf-Rayet and a Black Hole: ', len(id0_WRBH))
print('Binary systems that merge via GW: ', len(id0_GW))
print('Binary systems that evolve from a WR-BH and merge via GW: ', len(id0_both))

Binary systems of a Wolf-Rayet and a Black Hole:  130479
Binary systems that merge via GW:  340
Binary systems that evolve from a WR-BH and merge via GW:  306


### Mass transfer events

In [ ]:
#selecting ID of MT stable events (Roche Lobe)
df0_WRBH_stable = df0_WRBH.loc[df0_WRBH['BEvent']==5, ['ID','BEvent']].groupby('ID').count().reset_index() #reset index serve per avere ID come colonna e non come indice

In [ ]:
#selecting ID of MT unstable events (Common Envelop)
df0_WRBH_unstable = df0_WRBH.loc[(df0_WRBH['BEvent']==7) | (df0_WRBH['BEvent']==11), ['ID','BEvent']].groupby('ID').count().reset_index()

In [ ]:
#merging the two tables
mass_transfer0 = df0_WRBH_stable.merge(df0_WRBH_unstable, how='outer', on='ID')
mass_transfer0.rename(columns={'BEvent_x': 'MTEvents_stable', 'BEvent_y': 'MTEvents_unstable'})

#sustituting NA values with zeros
mass_transfer0['MTEvents_unstable'] = mass_transfer0['MTEvents_unstable'].fillna(0).astype(int)
mass_transfer0['MTEvents_stable'] = mass_transfer0['MTEvents_stable'].fillna(0).astype(int)

2024-05-15 18:47:40,941 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 675.41 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:47:40,920 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 676.45 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:47:41,446 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2024-05-15 18:47:47,600 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 03968c9ff7f80c9580f5e7d9d4e05239 initialized by task ('hash-join-transfer-03968c9ff7f80c9580f5e7d9d4e05239', 0) executed on worker tcp://127.0.0.1:33043
2024-05-15 18:57:53,029 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 0543e901a305484294131f58bdeb1c22 initialized by task ('hash-join-transfer-0543e901a305484294131f58bdeb1c22', 0) executed on worker tcp://127.0.0.1:46813
2024-05-15 18:57:53,113 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 8dc1925f8b4f5e7c08fe312bb95864f3 initialized by task ('hash-join-transfer-8dc1925f8b4f5e7c08fe312bb95864f3', 0) executed on worker tcp://127.0.0.1:34319
2024-05-15 18:57:53,925 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 0543e901a305484294131f58bdeb1c22 deactivated due to stimulus 'task-finished-1715792273.9198132'
2024-05-15 18:57:53,936 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 8dc1925f8b4f5e7c08fe312bb

,ID,MTEvents_stable,MTEvents_unstable,Mass_0,Mass_1,Semimajor,Eccentricity
0,7,NaN,NaN,18.225050,19.32260,NaN,NaN
1,11,NaN,NaN,21.816500,22.43766,2488.11,0.755503
2,33,NaN,NaN,1.531003,15.52529,NaN,NaN
3,115,NaN,NaN,20.162420,23.16621,NaN,NaN
4,129,NaN,NaN,19.957810,21.45724,NaN,NaN
...,...,...,...,...,...,...,...
130474,4180109,NaN,NaN,1.313853,14.98446,NaN,NaN
130475,4180149,NaN,NaN,22.531700,26.95646,18173.91,0.952452
130476,4180153,NaN,NaN,18.979820,22.57124,NaN,NaN
130477,4180212,NaN,NaN,12.622030,0.75723,NaN,NaN


In [ ]:
#merge stabel-unstable table with the table containing parameters of the binary system
data_0 = mass_transfer0.merge(data_WRBH, how='inner', on='ID')

#adding column which says if the system merge
col = data_0['ID'].isin(id0_both).astype(int)
data_0 = data_0.assign(Merge=col)

2024-05-15 18:59:31,056 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 670.97 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:59:35,012 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 672.61 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:59:35,606 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

['/home/gaia/LCP-B/project/out.csv/0.part']

In [ ]:
#saving the output
data_0.to_csv('./out.csv', index=False)  